In [623]:
# import the necessary packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [882]:
# File name containing details related to the positive samples (e.g. non-hallway)
pos_fpath = "/Users/debjani/Desktop/projects/Thesis/data/siamese_output/refrigarator/refrigarator_final.dat"


In [883]:
# File name containing details related to the negative samples
neg_fpath = "/Users/debjani/Desktop/projects/Thesis/data/siamese_output/nonrefrigarator/nonrefrigarator.dat"


In [884]:
pos_df = pd.read_csv(pos_fpath, delimiter = "\t")
pos_df = pos_df.drop_duplicates().reset_index(drop=True)
print(pos_df.shape)
pos_df.head(2)

(952, 8)


,Unnamed: 0,frame_id,xmin,ymin,width,height,score,ref_id
0,0,7509,1.0,1.000000,281.937799,270.0,56,7509
1,1,7511,1.0,1.954544,149.000000,258.0,58,7509


In [885]:
#ref_indices = [914,1930]
#ref_path = '/Users/debjani/Desktop/projects/Thesis/data/labelled_data/final_label/fire_extingusher_ref.dat'
#ref_df = pd.read_csv(ref_path, delimiter = "\t", header=None, names=['id', 'xmin', 'ymin', 'width', 'height'])
#print(ref_df.head())



In [886]:
# read ground truth file , with which output will be compared
gt_path = ref_path = '/Users/debjani/Desktop/projects/Thesis/data/labelled_data/final_label/refrigarator.dat'
gt_df = pd.read_csv(gt_path, delimiter = "\t", header=None, names=['id', 'xmin', 'ymin', 'width', 'height'])
gt_df.drop_duplicates().reset_index(drop=True)
print(gt_df.head())
print(gt_df.shape)


     id        xmin       ymin       width      height
0  7509  241.000000   7.000000  114.000000  114.000000
1  7511  241.915628   8.377973  114.113935  114.113935
2  7512  243.054053  11.128982  114.002001  114.002001
3  7513  243.633884  13.740451  113.949395  113.949395
4  7514  243.210160  15.973198  113.851163  113.851163
(953, 5)


In [887]:
neg_df = pd.read_csv(neg_fpath, delimiter = "\t", header=None, index_col=None, names = ['id', 'xmin', 'ymin', 'width', 'height', 'sim_score'])


In [888]:
#print(neg_df.head(57))
print(neg_df.shape)

(567, 6)


In [889]:
# Remove the duplicate frames from neg_df
neg_df = neg_df.drop_duplicates().reset_index(drop=True)
print(neg_df.shape)

(555, 6)


In [890]:
#print(neg_df.head(57))

In [891]:
# Defining the cut-off for positive and negative samples
sim_cutoff = 25
iou_cutoff = 0.3

In [892]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
       
    
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

In [893]:
neg_iou_vec = np.zeros((neg_df.shape[0], 1))
neg_df['iou'] = neg_iou_vec
for i in range(neg_iou_vec.shape[0]):
    #print(i)
    boxA = [0, 0, 0, 0]
    boxB = [0, 0, 0, 0]
    
    #print(neg_df['xmin'][i])
    boxA[0] = neg_df['xmin'][i]
    boxA[1] = neg_df['ymin'][i]
    boxA[2] = neg_df['xmin'][i] + neg_df['width'][i]
    boxA[3] = neg_df['ymin'][i] + neg_df['height'][i]
    boxB[0] = ref_df['xmin'][0]
    boxB[1] = ref_df['ymin'][0]
    boxB[2] = ref_df['xmin'][0] + gt_df['width'][0]
    boxB[3] = ref_df['ymin'][0] + gt_df['height'][0]
    neg_iou_vec[i] = bb_intersection_over_union(boxA, boxB)
    neg_df['iou'][i] = neg_iou_vec[i]
    #print(boxA)
    #print(boxB)

NameError: name 'ref_df' is not defined

In [894]:
#print(neg_df[neg_df['iou'] > iou_cutoff].head())

In [895]:
# Defining TN and FP
fp_df = neg_df[neg_df['sim_score'] >= sim_cutoff]
fp_df = fp_df[fp_df['iou'] >= iou_cutoff]
tn_df = pd.concat([neg_df, fp_df])
tn_df = tn_df.drop_duplicates(keep=False)

In [896]:
print(tn_df.shape)
print(fp_df.shape)
print(tn_df.head())

(555, 7)
(0, 7)
      id        xmin       ymin       width      height  sim_score  iou
0  809.0  257.111115   4.635177  142.306195  142.306195         22  0.0
1  812.0  273.695301   2.974011  152.529101  152.529101         19  0.0
2  813.0  285.555557  15.346902  142.306195  142.306195         17  0.0
3  814.0  221.869455   6.892373  127.372204  127.372204         17  0.0
4  815.0  290.279488   1.000000  190.720512  201.249414         19  0.0


In [897]:
#pos_df['xmin'] = pd.to_numeric(pos_df['xmin'])
iou_vec = np.zeros((pos_df.shape[0], 1))
for i in range(pos_df.shape[0]):
    
    boxA = [0, 0, 0, 0]
    boxB = [0, 0, 0, 0]
    
    boxA[0] = pos_df['xmin'][i]
    boxA[1] = pos_df['ymin'][i]
    boxA[2] = pos_df['xmin'][i] + pos_df['width'][i]
    boxA[3] = pos_df['ymin'][i] + pos_df['height'][i]
    
    ref_index = i #ref_indices.index(int(pos_df['ref_id'][i]))

    boxB[0] = gt_df['xmin'][ref_index]
    boxB[1] = gt_df['ymin'][ref_index]
    boxB[2] = gt_df['xmin'][ref_index] + gt_df['width'][ref_index]
    boxB[3] = gt_df['ymin'][ref_index] + gt_df['height'][ref_index]
    iou_vec[i] = bb_intersection_over_union(boxA, boxB)
    #print(iou_vec[i])

In [898]:
pos_df['iou'] = iou_vec
pos_df.head(69)

,Unnamed: 0,frame_id,xmin,ymin,width,height,score,ref_id,iou
0,0,7509,1.000000,1.000000,281.937799,270.000000,56,7509,0.058117
1,1,7511,1.000000,1.954544,149.000000,258.000000,58,7509,0.000000
2,2,7512,272.751748,1.000000,99.679105,172.598719,59,7509,0.469542
3,3,7513,272.751748,1.000000,99.679105,172.598719,58,7509,0.473775
4,4,7514,276.678963,1.000000,89.206532,154.465002,56,7509,0.523109
5,5,7515,276.678963,1.000000,89.206532,154.465002,57,7681,0.509679
6,6,7516,276.678963,1.000000,89.206532,154.465002,58,7681,0.499324
7,7,7517,272.751748,1.000000,99.679105,172.598719,58,7681,0.445548
8,8,7518,272.751748,1.000000,99.679105,172.598719,57,7681,0.445739
9,9,7519,272.751748,1.000000,99.679105,172.598719,57,7681,0.444351


In [899]:
#calculating Tp and Fn
tp_df = pos_df[pos_df['score'] >= sim_cutoff]
print(tp_df.shape)
tp_df = tp_df[tp_df['iou'] >= iou_cutoff]#
print(tp_df.shape)
fn_df = pd.concat([pos_df, tp_df])
fn_df = fn_df.drop_duplicates(keep=False)
print(tp_df.shape)

(952, 9)
(712, 9)
(712, 9)


In [900]:
print(tp_df.shape)
print(fn_df.shape)

(712, 9)
(240, 9)


In [901]:
# calcualting the counts
tp = float(tp_df.shape[0])
tn = tn_df.shape[0]
fp = fp_df.shape[0]
fn = fn_df.shape[0]
print(tp, fp, fn, tn)

(712.0, 0, 240, 555)


In [902]:
# Accuracy
accuracy = (tp+tn)/(tp+tn+fn+fp)
print(accuracy)

0.840743198407


In [903]:
# Precision
precision = tp/(tp+fp)
print(precision)

1.0


In [904]:
# Recall
recall = tp/(tp+fn)
print(recall)

0.747899159664


In [905]:
# F1
F1 = 2 * recall * precision/(precision + recall)
print(F1)

0.855769230769
